Regression Analysis of Environmental, Demographic, Health, and Poverty Data
1. Introduction
    This analysis examines relationships among environmental, demographic, health, and poverty metrics. The primary objectives are to:

    -Identify demographic and socioeconomic factors contributing to environmental pollution (PM2.5 levels).
    -Investigate the impact of poverty and demographic factors on health outcomes (e.g., coronary heart disease).
    -Assess the interplay between health, poverty, and environmental metrics to guide policy recommendations.

2. Dataset Overview
    The granularity of the dataset is at the census tract level. 
        Key variables analyzed:

            Demographics: Percentages of racial groups such as American Indian/Alaska Native, Asian, Black or African American, Hispanic or Latino, and White.
            Environmental Metrics: PM2.5 levels in the air and energy burden.
            Health Metrics: Prevalence of coronary heart disease and diabetes among adults aged 18 or older.
            Poverty Metrics: Percentage of individuals living below the poverty line, aggregated at the census tract level.

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.colors as colors
import matplotlib.patches as mpatches
from pandas import cut
from matplotlib import colormaps as cmap
import folium
from shapely.ops import unary_union


%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 8)
plt.style.use("ggplot")

C:\Users\Kassidi\AppData\Local\Temp\ipykernel_15772\392024889.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


3. Data Preprocessing
    The data was cleaned and prepared as follows:

        -Missing values were addressed by exclusion or imputation, depending on the variable's importance.
        -Continuous variables were scaled to ensure comparability across predictors.
        -Poverty variables were derived from raw census tract data and aggregated for analysis.

In [3]:
regression_cols = [
    'Census tract 2010 ID', 
    'County Name', 
    'State/Territory',
    'Total population', 
    'Percent American Indian / Alaska Native', 
    'Percent Asian', 'Percent Black or African American alone', 
    'Percent Hispanic or Latino', 
    'Percent Native Hawaiian or Pacific',
    'Percent other races', 
    'Percent White',
    'Energy burden',
    'Energy burden (percentile)',
    'Percent pre-1960s housing (lead paint indicator)',
    'Percent pre-1960s housing (lead paint indicator) (percentile)',
    'PM2.5 in the air',
    'PM2.5 in the air (percentile)',
    'Coronary heart disease among adults aged greater than or equal to 18 years', 
    'Coronary heart disease among adults aged greater than or equal to 18 years (percentile)', 
    'Current asthma among adults aged greater than or equal to 18 years', 
    'Current asthma among adults aged greater than or equal to 18 years (percentile)',
    'Diagnosed diabetes among adults aged greater than or equal to 18 years',
    'Diagnosed diabetes among adults aged greater than or equal to 18 years (percentile)',
    'Low life expectancy (percentile)', 
    'Percent age 10 to 64', 
    'Percent age over 64', 
    'Percent age under 10'
]   

reg_stats = pd.read_csv(r"C:\\New_499_Code\\499_Cleaned_Abbreviated_CEJST_Disadvantaged_Communities_Data.csv", usecols=regression_cols)

In [4]:
reg_stats.head()

,Census tract 2010 ID,County Name,State/Territory,Percent Black or African American alone,Percent American Indian / Alaska Native,Percent Asian,Percent Native Hawaiian or Pacific,Percent White,Percent Hispanic or Latino,Percent other races,...,PM2.5 in the air,Percent pre-1960s housing (lead paint indicator) (percentile),Percent pre-1960s housing (lead paint indicator),Current asthma among adults aged greater than or equal to 18 years (percentile),Current asthma among adults aged greater than or equal to 18 years,Diagnosed diabetes among adults aged greater than or equal to 18 years (percentile),Diagnosed diabetes among adults aged greater than or equal to 18 years,Coronary heart disease among adults aged greater than or equal to 18 years (percentile),Coronary heart disease among adults aged greater than or equal to 18 years,Low life expectancy (percentile)
0,1001020100,Autauga County,Alabama,0.07,0.0,0.00,0.00,0.83,0.01,0.00,...,9.15,41.0,17.0,57.0,990.0,60.0,1130.0,59.0,640.0,89.0
1,1001020200,Autauga County,Alabama,0.57,0.0,0.00,0.01,0.38,0.01,0.00,...,9.18,43.0,19.0,82.0,1100.0,83.0,1420.0,49.0,590.0,65.0
2,1001020300,Autauga County,Alabama,0.24,0.0,0.00,0.00,0.65,0.06,0.06,...,9.20,22.0,5.0,65.0,1019.0,66.0,1180.0,60.0,650.0,NaN
3,1001020400,Autauga County,Alabama,0.05,0.0,0.00,0.00,0.89,0.01,0.00,...,9.23,31.0,11.0,27.0,880.0,55.0,1080.0,66.0,680.0,77.0
4,1001020500,Autauga County,Alabama,0.18,0.0,0.03,0.00,0.70,0.04,0.00,...,9.24,1.0,0.0,37.0,919.0,34.0,919.0,31.0,500.0,41.0


4. Methods

    -Regression Models
    -Ordinary Least Squares (OLS) regression was employed.
    
    Models included:
        PM2.5 and demographic, health, and poverty metrics.
        Coronary heart disease and demographic, health, and poverty metrics.
        Energy burden and demographic, health, and poverty metrics.
        ANOVA
        Analysis of Variance (ANOVA) tested the significance of predictors in each model.

Correlation Analysis

    Correlation coefficients and a heatmap visualized relationships among environmental, health, and poverty variables.


Analysis

Regression Model 1: PM2.5 and Demographics, Health, and Poverty
Formula: Q("PM2.5 in the air") ~ Q("Percent White") + Q("Percent Black or African American alone") + Q("Percent Hispanic or Latino") + Q("Poverty rate") + Q("Diagnosed diabetes among adults aged >=18")

In [5]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [ ]:
#conduct ols logistic regression on PM2.5 and all race variables
#regression formula
model = ols('Q("PM2.5 in the air") ~ Q("Percent American Indian / Alaska Native") + Q("Percent Asian") + Q("Percent Black or African American alone") + Q("Percent Hispanic or Latino") + Q("Percent Native Hawaiian or Pacific") + Q("Percent other races") + Q("Percent White")', data=reg_stats).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     Q("PM2.5 in the air")   R-squared:                       0.223
Model:                               OLS   Adj. R-squared:                  0.223
Method:                    Least Squares   F-statistic:                     2951.
Date:                   Mon, 02 Dec 2024   Prob (F-statistic):               0.00
Time:                           01:30:03   Log-Likelihood:            -1.3305e+05
No. Observations:                  71903   AIC:                         2.661e+05
Df Residuals:                      71895   BIC:                         2.662e+05
Df Model:                              7                                         
Covariance Type:               nonrobust                                         
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                       13.7734      0.201     68.625      0.000      13.380      14.167
Q("Percent American Indian / Alaska Native")    -9.0261      0.245    -36.803      0.000      -9.507      -8.545
Q("Percent Asian")                              -0.8175      0.229     -3.573      0.000      -1.266      -0.369
Q("Percent Black or African American alone")    -5.0214      0.209    -24.013      0.000      -5.431      -4.612
Q("Percent Hispanic or Latino")                 -4.1318      0.206    -20.102      0.000      -4.535      -3.729
Q("Percent Native Hawaiian or Pacific")          0.8858      1.160      0.764      0.445      -1.387       3.158
Q("Percent other races")                         2.3298      0.097     24.111      0.000       2.140       2.519
Q("Percent White")                              -6.1648      0.210    -29.356      0.000      -6.576      -5.753
==============================================================================
Omnibus:                     6049.019   Durbin-Watson:                   0.146
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            13827.230
Skew:                           0.529   Prob(JB):                         0.00
Kurtosis:                       4.870   Cond. No.                         244.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
#conduct anova test on the model
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table


,sum_sq,df,F,PR(>F)
"Q(""Percent American Indian / Alaska Native"")",3210.672730,1.0,1354.471705,9.041215e-294
"Q(""Percent Asian"")",30.262124,1.0,12.766543,3.531051e-04
"Q(""Percent Black or African American alone"")",1366.820576,1.0,576.614296,6.482098e-127
"Q(""Percent Hispanic or Latino"")",957.841472,1.0,404.080166,1.257830e-89
"Q(""Percent Native Hawaiian or Pacific"")",1.383514,1.0,0.583656,4.448852e-01
"Q(""Percent other races"")",1377.976919,1.0,581.320771,6.253697e-128
"Q(""Percent White"")",2042.820214,1.0,861.795147,2.585764e-188
Residual,170421.659682,71895.0,NaN,NaN


## Model Overview
    Dependent Variable: PM2.5 in the air.
    R-squared: 0.223 (22.3% of the variation in PM2.5 levels is explained by the model's predictors).
    Adjusted R-squared: 0.223 (minimal difference from R-squared indicates the model doesn't overfit with the predictors included).
    F-statistic: 2951 (indicates the overall model is highly statistically significant, with a p-value of 0.00).
    Sample Size: 71,903 observations.
    Coefficients and Their Interpretations

1. Intercept (13.77)

    Represents the average PM2.5 level when all predictor variables are zero.
    Significant with a t-value of 68.625 and a p-value of 0.000.

2. Percent American Indian / Alaska Native (-9.03)
    A 1% increase in the population identifying as American Indian/Alaska Native is associated with a 9.03-unit decrease in PM2.5 levels, holding other variables constant.
    Highly significant with a p-value of 0.000.
3. Percent Asian (-0.82)
    A 1% increase in the population identifying as Asian is associated with a 0.82-unit decrease in PM2.5 levels.
    Statistically significant with a p-value of 0.000.
4. Percent Black or African American (-5.02)
    A 1% increase in the population identifying as Black or African American is associated with a 5.02-unit decrease in PM2.5 levels.
    Statistically significant with a p-value of 0.000.
5. Percent Hispanic or Latino (-4.13)
    A 1% increase in the population identifying as Hispanic or Latino is associated with a 4.13-unit decrease in PM2.5 levels.
    Statistically significant with a p-value of 0.000.
6. Percent Native Hawaiian or Pacific (0.89)
    A 1% increase in the population identifying as Native Hawaiian or Pacific Islander is associated with a 0.89-unit increase in PM2.5 levels.
    Not statistically significant with a p-value of 0.445.
7. Percent Other Races (2.33)
    A 1% increase in the population identifying as other races is associated with a 2.33-unit increase in PM2.5 levels.
    Highly significant with a p-value of 0.000.
8. Percent White (-6.16)
    A 1% increase in the population identifying as White is associated with a 6.16-unit decrease in PM2.5 levels.
    Highly significant with a p-value of 0.000.

## Model Diagnostics

1. Durbin-Watson (0.146)
    Indicates potential positive autocorrelation in residuals (values close to 0 suggest strong autocorrelation).

2. Jarque-Bera (JB): 13,827.230 (p=0.00)
    The residuals are not normally distributed, which may affect the validity of the coefficients.
    
3. Condition Number (244)
    A moderately high value indicates some potential multicollinearity among predictors, but it is not extreme.

## Key Takeaways
    Most predictors are statistically significant, with the exception of Percent Native Hawaiian or Pacific.
    The model suggests that certain racial demographics, particularly American Indian/Alaska Native, Asian, Black, Hispanic, and White populations, are associated with reduced PM2.5 levels.
    The small R-squared value (0.223) suggests that other factors not included in the model contribute to variations in PM2.5 levels.
    Multicollinearity and non-normal residuals may require further investigation to ensure the model's robustness.

In [8]:
#conduct multiple regression analysis on total population, pm2.5, and energy burden
model2 = ols('Q("PM2.5 in the air") ~ Q("Total population") + Q("Energy burden")', data=reg_stats).fit()
model2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     Q("PM2.5 in the air")   R-squared:                       0.009
Model:                               OLS   Adj. R-squared:                  0.008
Method:                    Least Squares   F-statistic:                     307.9
Date:                   Mon, 02 Dec 2024   Prob (F-statistic):          6.95e-134
Time:                           01:30:03   Log-Likelihood:            -1.4142e+05
No. Observations:                  71717   AIC:                         2.828e+05
Df Residuals:                      71714   BIC:                         2.829e+05
Df Model:                              2                                         
Covariance Type:               nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 8.4850      0.015    570.686      0.000       8.456       8.514
Q("Total population")  5.224e-05   2.81e-06     18.590      0.000    4.67e-05    5.77e-05
Q("Energy burden")       -0.0184      0.001    -14.722      0.000      -0.021      -0.016
==============================================================================
Omnibus:                    12363.720   Durbin-Watson:                   0.049
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            32382.867
Skew:                           0.948   Prob(JB):                         0.00
Kurtosis:                       5.691   Cond. No.                     1.16e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.16e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
#conduct anova test on the model
anova_table2 = sm.stats.anova_lm(model2, typ=2)
anova_table2

,sum_sq,df,F,PR(>F)
"Q(""Total population"")",1044.402098,1.0,345.605141,5.827703e-77
"Q(""Energy burden"")",654.995339,1.0,216.745788,5.468516e-49
Residual,216716.255764,71714.0,NaN,NaN


Model Overview:

    Dependent Variable: Coronary heart disease among adults aged greater than or equal to 18 years

    Independent Variables: Percentages of various racial/ethnic groups in the population.

    R-squared: 0.165

    This indicates that 16.5% of the variability in coronary heart disease prevalence is explained by the percentages of the racial/ethnic groups in the population.

    While this is relatively low, it suggests there may be other important factors influencing coronary heart disease prevalence that are not included in the model.
    
    Key Coefficients and Their Interpretation:
    Intercept: -115.22

    This represents the baseline coronary heart disease prevalence when all percentages for racial/ethnic groups are 0 (a theoretical value).
    While this has limited practical meaning, it provides the baseline for the regression equation.
    Q("Percent American Indian / Alaska Native"): 1231.96

    For every 1% increase in the percentage of American Indian / Alaska Native individuals, the prevalence of coronary heart disease increases by 1231.96 cases per unit population.
    P-value = 0.000: This is highly significant, indicating a strong relationship.
    Q("Percent Asian"): 140.27

    For every 1% increase in the percentage of Asian individuals, the prevalence of coronary heart disease increases by 140.27 cases per unit population.
    P-value = 0.000: Highly significant.
    Q("Percent Black or African American alone"): 894.34

    For every 1% increase in the percentage of Black or African American individuals, the prevalence of coronary heart disease increases by 894.34 cases per unit population.
    P-value = 0.000: Highly significant.
    Q("Percent Hispanic or Latino"): 662.91

    For every 1% increase in the percentage of Hispanic or Latino individuals, the prevalence of coronary heart disease increases by 662.91 cases per unit population.
    P-value = 0.000: Highly significant.
    Q("Percent Native Hawaiian or Pacific Islander"): 1482.93

    For every 1% increase in the percentage of Native Hawaiian or Pacific Islander individuals, the prevalence of coronary heart disease increases by 1482.93 cases per unit population.
    P-value = 0.000: Highly significant.
    Q("Percent other races"): -63.70

    For every 1% increase in the percentage of other races, the prevalence of coronary heart disease decreases by 63.70 cases per unit population.
    P-value = 0.000: Highly significant. This suggests that populations categorized as "other races" are associated with lower coronary heart disease prevalence.
    Q("Percent White"): 796.37

    For every 1% increase in the percentage of White individuals, the prevalence of coronary heart disease increases by 796.37 cases per unit population.
    P-value = 0.000: Highly significant.


    Below is the ranking of racial/ethnic groups by the strength of their association with coronary heart disease (highest to lowest, based on their coefficients):

Native Hawaiian or Pacific Islander: +1482.93 cases per 1% increase.

American Indian / Alaska Native: +1231.96 cases per 1% increase.

Black or African American: +894.34 cases per 1% increase.

White: +796.37 cases per 1% increase.

Hispanic or Latino: +662.91 cases per 1% increase.

Asian: +140.27 cases per 1% increase.

Other Races: -63.70 cases per 1% increase (negative association, indicating a decrease in coronary heart disease prevalence).

#### Native Hawaiian or Pacific Islander and American Indian / Alaska Native show the strongest positive associations with coronary heart disease prevalence.

#### Other Races is the only category with a negative association, suggesting that a higher percentage of this group in the population is associated with lower coronary heart disease prevalence.


In [10]:
#conduct multiple regression analysis on heart disease and race variables
model3 = ols('Q("Coronary heart disease among adults aged greater than or equal to 18 years") ~ Q("Percent American Indian / Alaska Native") + Q("Percent Asian") + Q("Percent Black or African American alone") + Q("Percent Hispanic or Latino") + Q("Percent Native Hawaiian or Pacific") + Q("Percent other races") + Q("Percent White")', data=reg_stats).fit()
model3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                                           OLS Regression Results                                                          
===========================================================================================================================================
Dep. Variable:     Q("Coronary heart disease among adults aged greater than or equal to 18 years")   R-squared:                       0.165
Model:                                                                                         OLS   Adj. R-squared:                  0.165
Method:                                                                              Least Squares   F-statistic:                     1989.
Date:                                                                             Mon, 02 Dec 2024   Prob (F-statistic):               0.00
Time:                                                                                     01:30:03   Log-Likelihood:            -4.6792e+05
No. Observations:                                                                            70301   AIC:                         9.359e+05
Df Residuals:                                                                                70293   BIC:                         9.359e+05
Df Model:                                                                                        7                                         
Covariance Type:                                                                         nonrobust                                         
================================================================================================================
                                                   coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                     -115.2199     23.991     -4.803      0.000    -162.242     -68.198
Q("Percent American Indian / Alaska Native")  1231.9625     29.370     41.947      0.000    1174.398    1289.527
Q("Percent Asian")                             140.2699     27.810      5.044      0.000      85.762     194.778
Q("Percent Black or African American alone")   894.3406     25.007     35.764      0.000     845.327     943.354
Q("Percent Hispanic or Latino")                662.9110     24.629     26.916      0.000     614.639     711.183
Q("Percent Native Hawaiian or Pacific")       1482.9297     77.305     19.183      0.000    1331.411    1634.448
Q("Percent other races")                       -63.7036     11.989     -5.313      0.000     -87.202     -40.205
Q("Percent White")                             796.3732     25.096     31.733      0.000     747.185     845.561
==============================================================================
Omnibus:                    10479.114   Durbin-Watson:                   0.874
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            33750.547
Skew:                           0.767   Prob(JB):                         0.00
Kurtosis:                       6.028   Cond. No.                         147.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
#conduct anova test on the model
anova_table3 = sm.stats.anova_lm(model3, typ=2)
anova_table3

,sum_sq,df,F,PR(>F)
"Q(""Percent American Indian / Alaska Native"")",6.226488e+07,1.0,1759.527267,0.000000e+00
"Q(""Percent Asian"")",9.002492e+05,1.0,25.439912,4.575108e-07
"Q(""Percent Black or African American alone"")",4.526142e+07,1.0,1279.030922,1.290090e-277
"Q(""Percent Hispanic or Latino"")",2.563755e+07,1.0,724.485067,9.095199e-159
"Q(""Percent Native Hawaiian or Pacific"")",1.302175e+07,1.0,367.978294,8.354875e-82
"Q(""Percent other races"")",9.990669e+05,1.0,28.232378,1.079164e-07
"Q(""Percent White"")",3.563477e+07,1.0,1006.993759,1.943724e-219
Residual,2.487478e+09,70293.0,NaN,NaN


In [12]:
#correlation matrix between population, pm2.5, and energy burden
correlation_matrix = reg_stats[['Total population', 'PM2.5 in the air', 'Energy burden']].corr()
correlation_matrix

,Total population,PM2.5 in the air,Energy burden
Total population,1.000000,0.075105,-0.090541
PM2.5 in the air,0.075105,1.000000,-0.061125
Energy burden,-0.090541,-0.061125,1.000000
